In [1]:
# 保存视频帧，按q退出
import cv2
import time
# 自己的摄像头为0，ip连接的摄像头为2
cap = cv2.VideoCapture(0)
i = 0
while(1):
    i += 1
    ret, frame = cap.read()
    img_gray=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    ret,img_thre=cv2.threshold(img_gray,132,255,cv2.THRESH_BINARY)
    cv2.imwrite('./tupian/'+ str(int(time.time()*1000000)) + '.png', frame)
    cv2.imshow('123',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [5]:
# 放在对应的数字文件夹下，新增卡片在words（‘黑色’）后面添加
words=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '(', ')', '+', '-', '/', '*',
          'A', 'B', 'C','D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
          'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '停车', '前进', '右转', '左转', '掉头',
          '橙色', '淡蓝色', '粉色', '红色', '绿色', '蓝色', '黄色', '黑色']
for i in range(len(words)):
    print('{}: {}'.format(i, words[i]))

0: 0
1: 1
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7
8: 8
9: 9
10: =
11: (
12: )
13: +
14: -
15: /
16: *
17: A
18: B
19: C
20: D
21: E
22: F
23: G
24: H
25: I
26: J
27: K
28: L
29: M
30: N
31: O
32: P
33: Q
34: R
35: S
36: T
37: U
38: V
39: W
40: X
41: Y
42: Z
43: 停车
44: 前进
45: 右转
46: 左转
47: 掉头
48: 橙色
49: 淡蓝色
50: 粉色
51: 红色
52: 绿色
53: 蓝色
54: 黄色
55: 黑色


In [1]:
# 卡片切割
import os
import cv2
import numpy as np
from imutils.perspective import four_point_transform

def f(img):
    row,col,c=img.shape
    img_gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    ret,img_thre=cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)
    kernel=np.ones([5,5])
    img_erode = cv2.erode(img_thre, kernel)
    img_dilate=cv2.dilate(img_erode,kernel)
    
    
    
    # 以下代码二选一注释掉，opencv版本不一样输出的参数不一样（可能输出两个参数或三个），观察有没有切割出图片。
    contours,hierarchy = cv2.findContours(img_dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#     _, contours,hierarchy = cv2.findContours(img_dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    
    count=0
    images=[]
    points=[]
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if x<10 or y<10  or (x+w)>(col-10) or (y+h)>(row-10):continue#边界过滤
        if abs(w/(h*1.0)-1)>0.3: continue#正方形过滤
        sclae=w*h/(row*col)
        if sclae<0.03 or sclae>0.8: continue#比例过滤
        if (w <= 50 or h <= 50): continue#大小过滤
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        warped = four_point_transform(img, box)#获取卡片

        temp=img[y:y+h,x:x+w,:]
        images.append(warped)
        points.append(box)
        count+=1
    if len(images)<=0:
        return None
    else:
        return images, points


directory_name = './tupian'
numk = 0
for filename in os.listdir(directory_name):
    try:
        frame = cv2.imread(directory_name + "/" + filename)
        cards, points = f(frame)
        if type(cards) != type(None):
            for i in range(len(cards)):
                card = cards[i]
                box = points[i]
                card = cv2.resize(card, (100, 100))
                numk += 1
                cv2.imwrite("./qiege/" + filename, card)
    except:
        pass


In [ ]:
# 然后手动将qiege目录下的图片进行筛选，模糊以及内容扭曲过大等删除，
# 参考之前kapianfenlei3文件夹下面的图片质量，然后在kapianfenlei3文件夹下创建对应数字的文件夹，将图片放入
# 最后调用“训练.ipynb”即可，需要修改总类别数。